In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movielens-20m-dataset/rating.csv
/kaggle/input/movielens-20m-dataset/link.csv
/kaggle/input/movielens-20m-dataset/genome_tags.csv
/kaggle/input/movielens-20m-dataset/genome_scores.csv
/kaggle/input/movielens-20m-dataset/tag.csv
/kaggle/input/movielens-20m-dataset/movie.csv


In [8]:
ratings = pd.read_csv('/kaggle/input/movielens-20m-dataset/rating.csv')
movies = pd.read_csv('/kaggle/input/movielens-20m-dataset/movie.csv')

In [9]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy


In [10]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [11]:
trainset, testset = train_test_split(data, test_size=0.2)

In [13]:
model = SVD()
model.fit(trainset)

In [14]:
predictions = model.test(testset)

In [15]:
rmse = accuracy.rmse(predictions)

RMSE: 0.7858


In [17]:
user_id = 1
user_movies = ratings[ratings['userId'] == user_id]['movieId']
user_unseen_movies = movies[~movies['movieId'].isin(user_movies)]

In [19]:
user_unseen_movies.loc[:,'predicted_rating'] = user_unseen_movies['movieId'].apply(lambda x: model.predict(user_id, x).est)
recommended_movies = user_unseen_movies.sort_values(by='predicted_rating', ascending=False).head(10)

In [20]:
print(recommended_movies[['movieId', 'title', 'predicted_rating']])

       movieId                                           title  \
18683    93006                   Very Potter Musical, A (2009)   
17799    89386                         Pearl Jam Twenty (2011)   
14592    72998                                   Avatar (2009)   
721        733                                Rock, The (1996)   
9539     27871                  Something the Lord Made (2004)   
13239    64716                             Seven Pounds (2008)   
20307    99813  Batman: The Dark Knight Returns, Part 2 (2013)   
12631    59315                                 Iron Man (2008)   
10286    34405                                 Serenity (2005)   
8975     26674                            Prime Suspect (1991)   

       predicted_rating  
18683          4.594398  
17799          4.485895  
14592          4.482812  
721            4.475665  
9539           4.464215  
13239          4.449667  
20307          4.415822  
12631          4.385487  
10286          4.383212  
8975         